In [19]:
import pandas as pd

In [20]:

pdf_output_header = pd.read_csv("data/output/header_output.csv")
pdf_output_detail = pd.read_csv("data/output/detail_output.csv")

In [21]:
pdf_output_header

,model_output_id,model_run_date,time_taken_seconds,features_used,model_type,split_year,version
0,4d6eb188-a612-4b1f-b400-ba1fce6cad07,2025-07-21 16:10:36.871603,0.896739,"['value_last_year', 'age_last_year']",XG_Boost,2023,v0.0.1
1,3e18eeb7-01b0-4e5e-8b30-1effa223f194,2025-07-21 16:10:40.977813,0.699201,"['value_last_year', 'age_from_peak']",Regression,2023,v0.0.1


In [22]:
pdf_output_detail

,model_output_id,player_id,name,year,age,predicted_value,actual_value
0,4d6eb188-a612-4b1f-b400-ba1fce6cad07,2857,Eldin Jakupovic,2023,38.0,0.212073,0.100
1,4d6eb188-a612-4b1f-b400-ba1fce6cad07,3333,James Milner,2023,37.0,1.326458,1.500
2,4d6eb188-a612-4b1f-b400-ba1fce6cad07,3755,Óscar Ustari,2023,36.0,0.750997,1.000
3,4d6eb188-a612-4b1f-b400-ba1fce6cad07,6893,Gabriel Tamas,2023,39.0,0.253268,0.100
4,4d6eb188-a612-4b1f-b400-ba1fce6cad07,8198,Cristiano Ronaldo,2023,38.0,21.124031,15.000
...,...,...,...,...,...,...,...
2629,3e18eeb7-01b0-4e5e-8b30-1effa223f194,996897,Diego Gómez,2023,20.0,1.291666,3.000
2630,3e18eeb7-01b0-4e5e-8b30-1effa223f194,1009658,Julio Soler,2023,18.0,0.407611,0.200
2631,3e18eeb7-01b0-4e5e-8b30-1effa223f194,1012534,Yankuba Minteh,2023,19.0,0.707873,2.000
2632,3e18eeb7-01b0-4e5e-8b30-1effa223f194,1031293,Ryan Oné,2023,17.0,0.170435,0.125


In [23]:
pdf_output = pdf_output_header.merge(pdf_output_detail, on="model_output_id")

In [24]:
pdf_output

,model_output_id,model_run_date,time_taken_seconds,features_used,model_type,split_year,version,player_id,name,year,age,predicted_value,actual_value
0,4d6eb188-a612-4b1f-b400-ba1fce6cad07,2025-07-21 16:10:36.871603,0.896739,"['value_last_year', 'age_last_year']",XG_Boost,2023,v0.0.1,2857,Eldin Jakupovic,2023,38.0,0.212073,0.100
1,4d6eb188-a612-4b1f-b400-ba1fce6cad07,2025-07-21 16:10:36.871603,0.896739,"['value_last_year', 'age_last_year']",XG_Boost,2023,v0.0.1,3333,James Milner,2023,37.0,1.326458,1.500
2,4d6eb188-a612-4b1f-b400-ba1fce6cad07,2025-07-21 16:10:36.871603,0.896739,"['value_last_year', 'age_last_year']",XG_Boost,2023,v0.0.1,3755,Óscar Ustari,2023,36.0,0.750997,1.000
3,4d6eb188-a612-4b1f-b400-ba1fce6cad07,2025-07-21 16:10:36.871603,0.896739,"['value_last_year', 'age_last_year']",XG_Boost,2023,v0.0.1,6893,Gabriel Tamas,2023,39.0,0.253268,0.100
4,4d6eb188-a612-4b1f-b400-ba1fce6cad07,2025-07-21 16:10:36.871603,0.896739,"['value_last_year', 'age_last_year']",XG_Boost,2023,v0.0.1,8198,Cristiano Ronaldo,2023,38.0,21.124031,15.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629,3e18eeb7-01b0-4e5e-8b30-1effa223f194,2025-07-21 16:10:40.977813,0.699201,"['value_last_year', 'age_from_peak']",Regression,2023,v0.0.1,996897,Diego Gómez,2023,20.0,1.291666,3.000
2630,3e18eeb7-01b0-4e5e-8b30-1effa223f194,2025-07-21 16:10:40.977813,0.699201,"['value_last_year', 'age_from_peak']",Regression,2023,v0.0.1,1009658,Julio Soler,2023,18.0,0.407611,0.200
2631,3e18eeb7-01b0-4e5e-8b30-1effa223f194,2025-07-21 16:10:40.977813,0.699201,"['value_last_year', 'age_from_peak']",Regression,2023,v0.0.1,1012534,Yankuba Minteh,2023,19.0,0.707873,2.000
2632,3e18eeb7-01b0-4e5e-8b30-1effa223f194,2025-07-21 16:10:40.977813,0.699201,"['value_last_year', 'age_from_peak']",Regression,2023,v0.0.1,1031293,Ryan Oné,2023,17.0,0.170435,0.125
